### RQ2 – Agent identity, change complexity, and merge probability

Goal of RQ2:

- We want to see **how PR size / complexity** (lines changed, files touched)
- and **which AI agent** (Codex, Copilot, Devin, Cursor, Claude Code)
- together affect the **probability that a pull request is merged**.

We will:

1. Load the PR table and the commit-detail table.
2. Build PR-level complexity features (total lines changed, files changed).
3. Join them back to get a `df_rq2` dataset.
4. Fit a **logistic regression** model to predict `merged_flag`.
5. Use **10-fold cross-validation** + a held-out test set to evaluate performance.
6. Interpret coefficients to answer RQ2.


In [1]:
import pandas as pd
import numpy as np

# scikit-learn core tools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

# Just for nicer console output
def create_divider():
    print('-' * 100)


### 1. Load main pull request table

First, we load the main `pull_request.parquet` table.  
This gives us PR-level metadata:

- PR id, agent name, state (open/closed), timestamps, repo id, etc.
- It **does not** contain complexity features like additions/deletions.


In [2]:
pull_requests = pd.read_parquet(
    'hf://datasets/hao-li/AIDev/pull_request.parquet'
)

print('pull_requests shape:', pull_requests.shape)
create_divider()
print('pull_requests columns:')
print(pull_requests.columns)
create_divider()

print(pull_requests.head())
create_divider()
pull_requests.info()


pull_requests shape: (33596, 14)
----------------------------------------------------------------------------------------------------
pull_requests columns:
Index(['id', 'number', 'title', 'body', 'agent', 'user_id', 'user', 'state',
       'created_at', 'closed_at', 'merged_at', 'repo_id', 'repo_url',
       'html_url'],
      dtype='object')
----------------------------------------------------------------------------------------------------
           id  number                                              title  \
0  3264933329    2911  Fix: Wait for all partitions in load_collectio...   
1  3265118634       2                   ファイルパス参照を相対パスに統一し、doc/からdocs/に統一   
2  3265640341      30        Add build staleness detection for debug CLI   
3  3265709660     205  feat: add comprehensive README screenshots wit...   
4  3265782173   17625        chore: remove HashedPostStateProvider trait   

                                                body        agent    user_id  \
0  ## Summary\n\

### 2. The PR table is missing complexity columns

The main PR table does **not** have:

- `additions`
- `deletions`
- `files_changed`

These fields live in the commit-related tables.  
To build PR-level complexity, we now load:

- `pr_commits.parquet`
- `pr_commit_details.parquet`


In [3]:
# Create a working copy of the pull_request table
prs = pull_requests.copy()

# Just to verify these features do NOT exist in this table
key_cols = ['pr_id', 'agent_name', 'additions', 'deletions', 'files_changed']
for c in key_cols:
    print(c, 'in columns?', c in prs.columns)

create_divider()

# Load commit-level tables
pr_commits = pd.read_parquet(
    'hf://datasets/hao-li/AIDev/pr_commits.parquet'
)

pr_commit_details = pd.read_parquet(
    'hf://datasets/hao-li/AIDev/pr_commit_details.parquet'
)

print('pr_commits columns:')
print(pr_commits.columns)
create_divider()

print('pr_commit_details columns:')
print(pr_commit_details.columns)
create_divider()


pr_id in columns? False
agent_name in columns? False
additions in columns? False
deletions in columns? False
files_changed in columns? False
----------------------------------------------------------------------------------------------------
pr_commits columns:
Index(['sha', 'pr_id', 'author', 'committer', 'message'], dtype='object')
----------------------------------------------------------------------------------------------------
pr_commit_details columns:
Index(['sha', 'pr_id', 'author', 'committer', 'message', 'commit_stats_total',
       'commit_stats_additions', 'commit_stats_deletions', 'filename',
       'status', 'additions', 'deletions', 'changes', 'patch'],
      dtype='object')
----------------------------------------------------------------------------------------------------


### 3. Clean PR table and define merge flag

We standardize column names and create a binary target:

- `pr_id` – PR identifier
- `agent_name` – which AI agent created this PR
- `merged_flag` – 1 if `merged_at` is non-null, 0 otherwise

`merged_flag` is our **target variable** for logistic regression.


In [4]:
prs = prs.rename(columns={
    'id': 'pr_id',        # PR identifier
    'agent': 'agent_name' # AI agent
})

# Merge flag: 1 if merged_at is not null, else 0
prs['merged_flag'] = prs['merged_at'].notna().astype(int)

print('=== PR-level columns (subset) ===')
print(prs[['pr_id', 'agent_name', 'state', 'created_at', 'merged_at', 'merged_flag']].head())

create_divider()
print('merged_flag value proportions:')
print(prs['merged_flag'].value_counts(normalize=True).round(3))
create_divider()


=== PR-level columns (subset) ===
        pr_id   agent_name   state            created_at  \
0  3264933329  Claude_Code  closed  2025-07-26T02:59:01Z   
1  3265118634  Claude_Code  closed  2025-07-26T04:56:55Z   
2  3265640341  Claude_Code  closed  2025-07-26T13:31:19Z   
3  3265709660  Claude_Code  closed  2025-07-26T14:07:22Z   
4  3265782173  Claude_Code    open  2025-07-26T15:02:48Z   

              merged_at  merged_flag  
0                  None            0  
1  2025-07-26T22:12:24Z            1  
2  2025-07-26T13:37:22Z            1  
3  2025-07-26T14:45:30Z            1  
4                  None            0  
----------------------------------------------------------------------------------------------------
merged_flag value proportions:
merged_flag
1    0.715
0    0.285
Name: proportion, dtype: float64
----------------------------------------------------------------------------------------------------


### 4. Build PR-level complexity features

Now we use `pr_commit_details` to build complexity for each PR.

For each `pr_id`, we compute:

- `total_additions` – sum of additions across all files
- `total_deletions` – sum of deletions
- `files_changed` – number of unique files touched
- `total_changes` – sum of `changes` across files (lines changed)

These become our **complexity features**.


In [5]:
# Work from pr_commit_details
cd = pr_commit_details.copy()

print('=== commit_details sample ===')
print(cd.head())
create_divider()

# Aggregate per PR
complexity = (
    cd.groupby('pr_id')
      .agg(
          total_additions=('additions', 'sum'),
          total_deletions=('deletions', 'sum'),
          files_changed=('filename', 'nunique'),
          total_changes=('changes', 'sum'),
      )
      .reset_index()
)

print('=== Complexity features (per PR) ===')
print(complexity.head())
create_divider()

# Replace any remaining NaNs with 0 (safe for these counts)
complexity[['total_additions', 'total_deletions', 'files_changed', 'total_changes']] = (
    complexity[['total_additions', 'total_deletions', 'files_changed', 'total_changes']]
    .fillna(0)
)


=== commit_details sample ===
                                        sha       pr_id  \
0  2f9d54dda4f0c87c19e0bbeb9936f525d0587e16  3271196926   
1  2f9d54dda4f0c87c19e0bbeb9936f525d0587e16  3271196926   
2  2f9d54dda4f0c87c19e0bbeb9936f525d0587e16  3271196926   
3  dbd1b5f129f7cffa5ce284d7255814c98bcc38a2  3271196926   
4  c2659cfdedf666c8f14753d71664563c2a932b23  3271196926   

                      author                  committer  \
0  devin-ai-integration[bot]  devin-ai-integration[bot]   
1  devin-ai-integration[bot]  devin-ai-integration[bot]   
2  devin-ai-integration[bot]  devin-ai-integration[bot]   
3  devin-ai-integration[bot]  devin-ai-integration[bot]   
4  devin-ai-integration[bot]  devin-ai-integration[bot]   

                                             message  commit_stats_total  \
0  Add llms.txt compilation system for AI model d...               23008   
1  Add llms.txt compilation system for AI model d...               23008   
2  Add llms.txt compilation syst

### 5. Check missing values in PR table

Some columns in `prs` have missing values, e.g. `merged_at` and `closed_at`.
This is **expected**:

- `merged_at` is missing when a PR was never merged.
- We already encoded this into `merged_flag`.

For RQ2, we only care that:

- `pr_id`, `agent_name`, `merged_flag` are not missing.


In [6]:
print('Missing values in prs:')
print(prs.isna().sum().sort_values(ascending=False))
create_divider()

print('Proportion missing per column (%):')
print((prs.isna().mean() * 100).round(2).sort_values(ascending=False))
create_divider()


Missing values in prs:
merged_at      9582
closed_at      2312
body            360
title             0
agent_name        0
number            0
pr_id             0
user              0
user_id           0
created_at        0
state             0
repo_id           0
repo_url          0
html_url          0
merged_flag       0
dtype: int64
----------------------------------------------------------------------------------------------------
Proportion missing per column (%):
merged_at      28.52
closed_at       6.88
body            1.07
title           0.00
agent_name      0.00
number          0.00
pr_id           0.00
user            0.00
user_id         0.00
created_at      0.00
state           0.00
repo_id         0.00
repo_url        0.00
html_url        0.00
merged_flag     0.00
dtype: float64
----------------------------------------------------------------------------------------------------


### 6. Merge PR metadata with complexity and create log features

We now join:

- `prs` (PR metadata + agent + merged_flag)
- `complexity` (per-PR size metrics)

Then:

- Fill missing `total_changes` and `files_changed` with 0 (for safety).
- Create `log_total_changes = log(1 + total_changes)` to reduce skew.
- Drop rows where key variables are missing.


In [7]:
# Merge PR metadata with complexity features
df_rq2 = prs.merge(complexity, on='pr_id', how='inner')

print('df_rq2 shape:', df_rq2.shape)
create_divider()
print(df_rq2[['pr_id', 'agent_name', 'merged_flag', 'total_changes', 'files_changed']].head())
create_divider()

# Basic cleaning
df_rq2['total_changes'] = df_rq2['total_changes'].fillna(0)
df_rq2['files_changed'] = df_rq2['files_changed'].fillna(0)

# Log-transform complexity
df_rq2['log_total_changes'] = np.log1p(df_rq2['total_changes'])

# Drop any rows missing in model-relevant columns
df_rq2 = df_rq2.dropna(
    subset=['agent_name', 'merged_flag', 'log_total_changes', 'files_changed']
)


df_rq2 shape: (33580, 19)
----------------------------------------------------------------------------------------------------
        pr_id   agent_name  merged_flag  total_changes  files_changed
0  3264933329  Claude_Code            0          396.0              3
1  3265118634  Claude_Code            1           76.0             11
2  3265640341  Claude_Code            1          407.0              5
3  3265709660  Claude_Code            1          300.0             15
4  3265782173  Claude_Code            0          221.0             21
----------------------------------------------------------------------------------------------------


In [8]:
print('Missing values in key columns:')
print(df_rq2[['agent_name', 'merged_flag', 'log_total_changes', 'files_changed']].isna().sum())
create_divider()

print('merged_flag distribution (proportion):')
print(df_rq2['merged_flag'].value_counts(normalize=True).round(3))
create_divider()

agent_summary = (
    df_rq2.groupby('agent_name')
    .agg(
        n_prs=('pr_id', 'nunique'),
        mean_total_changes=('total_changes', 'mean'),
        median_total_changes=('total_changes', 'median'),
        merge_rate=('merged_flag', 'mean'),
    )
    .sort_values('n_prs', ascending=False)
)

print('Agent-level summary for RQ2:')
print(agent_summary)
create_divider()


Missing values in key columns:
agent_name           0
merged_flag          0
log_total_changes    0
files_changed        0
dtype: int64
----------------------------------------------------------------------------------------------------
merged_flag distribution (proportion):
merged_flag
1    0.715
0    0.285
Name: proportion, dtype: float64
----------------------------------------------------------------------------------------------------
Agent-level summary for RQ2:
              n_prs  mean_total_changes  median_total_changes  merge_rate
agent_name                                                               
OpenAI_Codex  21793          800.888037                  74.0    0.826137
Copilot        4967         5538.911013                 179.0    0.430642
Devin          4822         2680.778308                 167.0    0.538158
Cursor         1540         2861.625325                 271.5    0.652597
Claude_Code     458         8646.951965                 691.0    0.591703
---------

### 7. Logistic regression setup

Now we model the probability that a PR is merged.

Target:

- `merged_flag` (1 = merged, 0 = not merged)

Features:

- Numeric: `log_total_changes`, `files_changed`
- Categorical: `agent_name`

We will:

1. Split into train/test (80/20).
2. Build a scikit-learn pipeline with:
   - `StandardScaler` for numeric features
   - `OneHotEncoder` for `agent_name`
   - `LogisticRegression` with `class_weight="balanced"`
3. Evaluate with:
   - 10-fold stratified cross-validation (ROC AUC)
   - Test-set ROC AUC, confusion matrix, and classification report.


In [9]:
# Define feature and target columns
feature_num_cols = ['log_total_changes', 'files_changed']
feature_cat_cols = ['agent_name']

# Make sure these columns exist
for column in feature_num_cols + feature_cat_cols + ['merged_flag']:
    assert column in df_rq2.columns, f'Missing column: {column}'

X = df_rq2[feature_num_cols + feature_cat_cols]
y = df_rq2['merged_flag']

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(f'Train size: {len(x_train)}, Test size: {len(x_test)}')
create_divider()


Train size: 26864, Test size: 6716
----------------------------------------------------------------------------------------------------


In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_val_score

numeric_features = feature_num_cols
categorical_features = feature_cat_cols

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
    ]
)

log_regression = LogisticRegression(
    max_iter=1000,
    class_weight='balanced',
    solver='lbfgs'
)

clf = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('classifier', log_regression),
    ]
)

# 10-fold stratified cross-validation (ROC AUC)
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

cv_auc = cross_val_score(
    clf,
    X,
    y,
    cv=cv,
    scoring='roc_auc'
)

print(f'10-fold CV ROC AUC scores: {cv_auc}')
print(f'Mean CV AUC: {cv_auc.mean():.3f} +/- {cv_auc.std():.3f}')
create_divider()


10-fold CV ROC AUC scores: [0.71810052 0.71641148 0.71209771 0.71361528 0.71502317 0.71532107
 0.71422808 0.72443705 0.72054952 0.71207789]
Mean CV AUC: 0.716 +/- 0.004
----------------------------------------------------------------------------------------------------


In [11]:
# Fit pipeline on training data
clf.fit(x_train, y_train)

# Evaluate on test data
y_pred_proba = clf.predict_proba(x_test)[:, 1]
y_pred = clf.predict(x_test)

test_auc = roc_auc_score(y_test, y_pred_proba)
cm = confusion_matrix(y_test, y_pred)

print('\n=== Test set performance ===')
print('Test ROC AUC:', round(test_auc, 3))
print('\nConfusion matrix (rows=true, cols=pred):')
print(cm)
print('\nClassification report:')
print(classification_report(y_test, y_pred))

create_divider()

# Inspect coefficients (feature importance)
log_reg_model = clf.named_steps['classifier']
ohe = clf.named_steps['preprocessor'].named_transformers_['cat']

# Get feature names: numeric + one-hot categorical
numeric_names = feature_num_cols
cat_ohe_names = ohe.get_feature_names_out(feature_cat_cols).tolist()
all_feature_names = numeric_names + cat_ohe_names

coef = log_reg_model.coef_[0]

coef_df = pd.DataFrame({
    'feature': all_feature_names,
    'coef': coef,
    'odds_ratio': np.exp(coef)
}).sort_values('coef', ascending=False)

print('=== Coefficients sorted by effect on merge odds ===')
print(coef_df)
create_divider()



=== Test set performance ===
Test ROC AUC: 0.717

Confusion matrix (rows=true, cols=pred):
[[1161  752]
 [1195 3608]]

Classification report:
              precision    recall  f1-score   support

           0       0.49      0.61      0.54      1913
           1       0.83      0.75      0.79      4803

    accuracy                           0.71      6716
   macro avg       0.66      0.68      0.67      6716
weighted avg       0.73      0.71      0.72      6716

----------------------------------------------------------------------------------------------------
=== Coefficients sorted by effect on merge odds ===
                   feature      coef  odds_ratio
6  agent_name_OpenAI_Codex  0.936027    2.549830
4        agent_name_Cursor  0.136502    1.146257
1            files_changed  0.047418    1.048560
2   agent_name_Claude_Code -0.084469    0.919000
0        log_total_changes -0.217280    0.804704
5         agent_name_Devin -0.414107    0.660930
3       agent_name_Copilot -0.8821

### 8. RQ2 conclusion (plain language)

**Model performance**

- 10-fold stratified CV AUC ≈ **0.716 ± 0.004**
- Test-set ROC AUC ≈ **0.717**

This means:

- If we randomly pick one merged PR and one non-merged PR,
- the model gives the merged one a higher score about **71–72%** of the time.
- So **change complexity + agent identity** carry real signal about merge probability.

**Effect of complexity**

- `log_total_changes` has a **negative coefficient** (odds ratio ≈ 0.80).
- Interpretation: for a 1 standard deviation increase in log total changes,
  the **odds of being merged drop by about 20%**, holding agent identity fixed.
- `files_changed` has a very small effect (odds ratio ≈ 1.05), so once we know how many lines changed, the number of files does not matter much.

**Effect of agent identity**

After controlling for complexity:

- **OpenAI Codex** PRs have much higher merge odds (odds ratio ≈ 2.55).
- **Cursor** has a mild advantage (odds ratio ≈ 1.15).
- **Claude Code** is roughly neutral (odds ratio ≈ 0.92).
- **Devin** PRs have lower merge odds (odds ratio ≈ 0.66).
- **Copilot** has the lowest merge odds (odds ratio ≈ 0.41).

**Answer to RQ2**

- **Yes**, change complexity and agent identity both matter.
- Larger, more complex PRs are **less likely to be merged**.
- Even at similar complexity levels, different AI agents see **different acceptance patterns**: Codex PRs are favored, while Copilot and Devin PRs are more likely to be rejected.
